In [1]:
from dataset import AudioDataset
from torch.utils.data import DataLoader
from model2 import EmotionRecognizer
import torch.nn

/Users/zuzia/Desktop/uni/KU/multimodality/venv/lib/python3.7/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
train_annotations = '/Users/zuzia/Downloads/MELD.Raw/train/train_sent_emo.csv'
val_annotations = '/Users/zuzia/Downloads/MELD.Raw/dev_sent_emo.csv'

train_audio = '/Users/zuzia/Downloads/MELD.Raw/train/train_splits/wav'
val_audio = '/Users/zuzia/Downloads/MELD.Raw/dev_splits_complete/wav'


In [3]:
train = AudioDataset(train_annotations, train_audio)
val = AudioDataset(val_annotations, val_audio)

train_dataloader = DataLoader(train, batch_size=32, shuffle = True)
val_dataloader = DataLoader(val, batch_size=32, shuffle=False)

train_size = len(train)

In [4]:
# Sanity check: Size should be [batch_size, num_features, feature_vector_len], [batch_size]
# Where batch size is 32, num_features (num mfccs) is 40 
train_features, train_labels = next(iter(train_dataloader))
print(f"Feature batch shape: {train_features.size()}")
print(f"Labels batch shape: {train_labels.size()}")
features = train_features[0]
label = train_labels[0]


Feature batch shape: torch.Size([32, 48])
Labels batch shape: torch.Size([32])


In [5]:
model = EmotionRecognizer(in_feat=train.num_features, num_classes=train.num_classes, p_dropout=0.3, lr=1e-4)
print(model)
print('num params:', model.count_parameters())

EmotionRecognizer(
  (conv): Sequential(
    (0): Conv1d(32, 1, kernel_size=(9,), stride=(2,), padding=(4,))
    (1): BatchNorm1d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): Dropout(p=0.3, inplace=False)
    (4): Conv1d(32, 64, kernel_size=(7,), stride=(2,), padding=(3,))
    (5): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (6): ReLU()
    (7): Dropout(p=0.3, inplace=False)
    (8): Conv1d(64, 128, kernel_size=(5,), stride=(2,), padding=(2,))
    (9): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (10): ReLU()
    (11): Dropout(p=0.3, inplace=False)
  )
  (head): Sequential(
    (0): Flatten(start_dim=1, end_dim=-1)
    (1): Linear(in_features=768, out_features=7, bias=True)
  )
  (criterion): CrossEntropyLoss()
  (accuracy): Accuracy()
)
num params: 61608


In [6]:
for batch in train_dataloader:
    x, y = batch
    print(x.shape, y.shape)
    break

torch.Size([32, 48]) torch.Size([32])


In [7]:
h = model(x) 

ValueError: Expected more than 1 value per channel when training, got input size torch.Size([1, 24])